In [1]:
# Camila Losada - 06/02/2025

### Example on how to use NeuronData structure
Define functions to process populations in various useful ways.

* The idea is that they receive a list of dictionaries as input. Each dictionary contains the necessary parameters to align spikes to a specific event.
* It can also receive attributes as input that should be removed from the neuron object (the information is deleted, but the attribute itself is not removed).

In [2]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
from typing import Dict, List
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.structures.population_data import PopulationData
import pandas as pd
import glob
import platform

In [3]:
if platform.system() == 'Linux':
    basepath = '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure'
elif platform.system() == 'Windows':
    basepath = '//envau_cifs.intlocal.univ-amu.fr/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure'

In [4]:
neu_path = basepath + "/session_struct/lip/neurons/*neu.h5"
path_list = glob.glob(neu_path) # get the path of all neurons inside the folder

In [5]:
# Define the function to read and preprocess each neuron 
def read_and_compute(path,params,rf_loc_df):
    neu=NeuronData.from_python_hdf5(path)
    neu=neu.get_neu_align(params=params,delete_att=['sp_samples'],rf_loc_df=rf_loc_df)
    return neu

In [6]:
# Define the events to align the spikes
params = [
    {
        "loc": "in",
        "event": "sample_on",
        "time_before": 500,
        "time_after": 1000,
        "select_block": 1,
    },
    {
        "loc": "in",
        "event": "test_on_1",
        "time_before": 500,
        "time_after": 600,
        "select_block": 1,
    },
    {
        "loc": "out",
        "event": "sample_on",
        "time_before": 500,
        "time_after": 1000,
        "select_block": 1,
    },
    {
        "loc": "out",
        "event": "test_on_1",
        "time_before": 500,
        "time_after": 600,
        "select_block": 1,
    }
]

In [7]:
rf_loc_df=pd.read_csv("//envau_cifs.intlocal.univ-amu.fr/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/activation_index/rf_loc_df_lip.csv")

In [9]:
# Preprocess all neurons
population = Parallel(n_jobs=-1)(
    delayed(read_and_compute)(path,params,rf_loc_df) for path in tqdm(path_list[:2])
)

100%|██████████| 2/2 [00:00<00:00, 122.94it/s]


In [10]:
# Create population object
popu = PopulationData(population)

In [17]:
popu.__dict__

{'population': [<ephysvibe.structures.neuron_data.NeuronData at 0x173f8adafd0>,
 'comment': '',
 '_PopulationData__created': '06/02/2025 11:06:13'}

In [19]:
i_neu = 0
popu.population[i_neu].__dict__.keys()

dict_keys(['date_time', 'subject', 'area', 'experiment', 'recording', 'sp_samples', 'cluster_id', 'cluster_ch', 'cluster_group', 'cluster_number', 'cluster_array_pos', 'cluster_depth', 'block', 'trial_error', 'code_samples', 'code_numbers', 'position', 'pos_code', 'sample_id', 'test_stimuli', 'test_distractor', 'rf_loc', 'sp_son_in', 'mask_son_in', 'time_before_son_in', 'sp_t1on_in', 'mask_t1on_in', 'time_before_t1on_in', 'sp_son_out', 'mask_son_out', 'time_before_son_out', 'sp_t1on_out', 'mask_t1on_out', 'time_before_t1on_out'])

In [20]:
popu.population[i_neu].sp_son_in

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [21]:
# Save population
popu.to_python_hdf5('population.h5')
# Read population
popu = PopulationData.from_python_hdf5('population.h5')

In [22]:
popu.population[i_neu].sp_son_in

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)